# Flow ML Course

## TP 1 : Vertex AI Experiments

L'objectif de ce notebook est d'apprendre à utiliser les experiments de Flow ML.

Les Experiments permettent de logger les méta paramètres des architectures des modèles et leurs métriques. On peut regrouper les différents entrainements sous une experiment pour les comparer.


En savoir plus [Experiments]( https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).

En savoir plus [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

Ce premier TP se déroulera intégralement sur la plateforme.

Votre Objectif sera de déterminer quel est les hyper paramètres qui permettent d'avoir le meilleur modèle.

In [ ]:
best_set_hparams = {...}

## Intialisation

In [4]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

BUCKET_URI = "gs://cours1bucket"

SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" 

! gsutil mb -l $LOCATION $BUCKET_URI

In [9]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

Pour ce TP nous allons utiliser le SDK python de Vertex AI. L'objet central de ce module est : aiplatform

Initialisons le :

In [10]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Introduction to Vertex AI Experiments PROF



In [24]:
# Specify a name for the experiment
EXPERIMENT_NAME = "supaero-vertex-course-1-1"

if EXPERIMENT_NAME == "[your-experiment-name]":
    EXPERIMENT_NAME = f"example-{uuid.uuid1()}"

In [ ]:
import random

units_choice = [32, 64, 128, 256]
epochs_choice = [1, 10, 100, 1000]
batch_size_choice = [1, 8, 32, 64]
learning_rate_choice = [.0001, .01, .1, 1]

# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)

for i in range(1, 11):
    aiplatform.start_run(f"run-{str(i)}")

    metaparams = {}
    metaparams["units"] = random.choice(units_choice)
    aiplatform.log_params(metaparams)

    hyperparams = {}
    hyperparams["epochs"] = random.choice(epochs_choice)
    hyperparams["batch_size"] = random.choice(batch_size_choice)
    hyperparams["learning_rate"] = random.choice(learning_rate_choice)
    aiplatform.log_params(hyperparams)

    metrics = {}
    metrics["test_acc"] = random.random() * 30 + 68
    metrics["train_acc"] = min(random.random() * 3 + metrics["test_acc"], 100)

    aiplatform.log_metrics(metrics)
    
    aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()


In [ ]:
experiment_df

#### Résultats Experiment


When you are finished with a run within an experiment, call `end_run()` method to complete the logging for that run.

Next, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

#### Delete the experiment

Next, delete the experiment using the `delete()` method.

In [ ]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)